In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier, CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

# Classification

### Подготовка данных

In [2]:
df = pd.read_csv('data/students_preprocessed.csv', sep = ',')
students_df = df[list(('school', 'sex', 'age', 'Pstatus', 'studytime', \
                                  'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'higher', \
                                  'internet', 'absences', 'G1', 'G2', 'G3'))]

In [3]:
X = students_df.drop('G3', axis = 1)
y = students_df.G3

In [35]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = 0.15)

In [36]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
def print_classification_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test, y_test))

### DTC (base model)

In [38]:
%%time
tree_params = {
    'max_depth': range(1, 17, 2),
    'min_samples_leaf': range(1, 8),
    'min_samples_split': range(2, 10, 2)
}

tree = DecisionTreeClassifier(random_state=0)
tree_base = GridSearchCV(tree, tree_params, cv=5).fit(X_train, y_train)
tree_base.best_params_

Wall time: 5.05 s


{'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [39]:
print_classification_metrics(tree_base, y_test, tree_base.predict(X_test))

[[5 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 2 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 4 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 4 0 0 0 0 0 0 0]
 [1 0 0 0 0 6 2 1 0 0 0 0 0]
 [0 0 0 0 0 0 4 1 1 0 0 0 0]
 [0 0 0 0 0 0 1 2 1 1 0 0 0]
 [0 0 0 0 0 0 2 0 3 1 0 0 0]
 [0 0 0 0 0 0 0 0 2 1 1 0 0]
 [0 0 0 0 0 0 0 0 2 1 2 2 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]]
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           5       0.00      0.00      0.00         1
           6       0.50      0.67      0.57         3
           8       0.80      0.67      0.73         6
           9       0.00      0.00      0.00         5
          10       0.55      0.60      0.57        10
          11       0.44      0.67      0.53         6
          12       0.50      0.40      0.44         5
          13       0.33      0.50      0.40         6
          14       0.25      0.25      0.25         4
          15       0.50    

### CatBoostClassifier

In [40]:
%%time
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train, y_train)

0:	learn: 2.6971668	total: 28.5ms	remaining: 28.5s
1:	learn: 2.5983237	total: 54.8ms	remaining: 27.3s
2:	learn: 2.5135843	total: 81.4ms	remaining: 27.1s
3:	learn: 2.4057829	total: 108ms	remaining: 26.8s
4:	learn: 2.2773386	total: 138ms	remaining: 27.4s
5:	learn: 2.1944125	total: 163ms	remaining: 27s
6:	learn: 2.1257177	total: 193ms	remaining: 27.4s
7:	learn: 2.0575976	total: 219ms	remaining: 27.1s
8:	learn: 1.9901780	total: 237ms	remaining: 26.1s
9:	learn: 1.9443186	total: 262ms	remaining: 25.9s
10:	learn: 1.8805055	total: 288ms	remaining: 25.9s
11:	learn: 1.8232324	total: 321ms	remaining: 26.4s
12:	learn: 1.7807047	total: 348ms	remaining: 26.4s
13:	learn: 1.7296678	total: 377ms	remaining: 26.6s
14:	learn: 1.6869255	total: 404ms	remaining: 26.6s
15:	learn: 1.6432249	total: 427ms	remaining: 26.2s
16:	learn: 1.6139829	total: 453ms	remaining: 26.2s
17:	learn: 1.5838316	total: 482ms	remaining: 26.3s
18:	learn: 1.5503980	total: 506ms	remaining: 26.1s
19:	learn: 1.5261094	total: 541ms	remain

164:	learn: 0.3479284	total: 4.33s	remaining: 21.9s
165:	learn: 0.3454332	total: 4.36s	remaining: 21.9s
166:	learn: 0.3442405	total: 4.39s	remaining: 21.9s
167:	learn: 0.3424972	total: 4.42s	remaining: 21.9s
168:	learn: 0.3394236	total: 4.44s	remaining: 21.9s
169:	learn: 0.3373848	total: 4.47s	remaining: 21.8s
170:	learn: 0.3354360	total: 4.49s	remaining: 21.8s
171:	learn: 0.3329867	total: 4.52s	remaining: 21.8s
172:	learn: 0.3308576	total: 4.55s	remaining: 21.8s
173:	learn: 0.3287581	total: 4.58s	remaining: 21.7s
174:	learn: 0.3269839	total: 4.61s	remaining: 21.7s
175:	learn: 0.3246639	total: 4.63s	remaining: 21.7s
176:	learn: 0.3217346	total: 4.66s	remaining: 21.7s
177:	learn: 0.3189569	total: 4.68s	remaining: 21.6s
178:	learn: 0.3167261	total: 4.72s	remaining: 21.6s
179:	learn: 0.3148145	total: 4.75s	remaining: 21.7s
180:	learn: 0.3132912	total: 4.78s	remaining: 21.6s
181:	learn: 0.3117801	total: 4.8s	remaining: 21.6s
182:	learn: 0.3100761	total: 4.83s	remaining: 21.6s
183:	learn: 0

323:	learn: 0.1600282	total: 8.55s	remaining: 17.8s
324:	learn: 0.1595627	total: 8.57s	remaining: 17.8s
325:	learn: 0.1589088	total: 8.6s	remaining: 17.8s
326:	learn: 0.1584988	total: 8.63s	remaining: 17.8s
327:	learn: 0.1576064	total: 8.65s	remaining: 17.7s
328:	learn: 0.1571437	total: 8.68s	remaining: 17.7s
329:	learn: 0.1564276	total: 8.7s	remaining: 17.7s
330:	learn: 0.1559507	total: 8.73s	remaining: 17.6s
331:	learn: 0.1551731	total: 8.76s	remaining: 17.6s
332:	learn: 0.1544783	total: 8.78s	remaining: 17.6s
333:	learn: 0.1539314	total: 8.8s	remaining: 17.6s
334:	learn: 0.1531794	total: 8.82s	remaining: 17.5s
335:	learn: 0.1528354	total: 8.85s	remaining: 17.5s
336:	learn: 0.1521261	total: 8.87s	remaining: 17.4s
337:	learn: 0.1514635	total: 8.89s	remaining: 17.4s
338:	learn: 0.1506382	total: 8.91s	remaining: 17.4s
339:	learn: 0.1499183	total: 8.94s	remaining: 17.4s
340:	learn: 0.1494259	total: 8.96s	remaining: 17.3s
341:	learn: 0.1488607	total: 8.98s	remaining: 17.3s
342:	learn: 0.1

482:	learn: 0.0991532	total: 12.7s	remaining: 13.6s
483:	learn: 0.0989876	total: 12.7s	remaining: 13.6s
484:	learn: 0.0988268	total: 12.8s	remaining: 13.6s
485:	learn: 0.0985446	total: 12.8s	remaining: 13.5s
486:	learn: 0.0983122	total: 12.8s	remaining: 13.5s
487:	learn: 0.0980063	total: 12.8s	remaining: 13.5s
488:	learn: 0.0976610	total: 12.9s	remaining: 13.4s
489:	learn: 0.0973139	total: 12.9s	remaining: 13.4s
490:	learn: 0.0970501	total: 12.9s	remaining: 13.4s
491:	learn: 0.0967748	total: 12.9s	remaining: 13.4s
492:	learn: 0.0966175	total: 13s	remaining: 13.3s
493:	learn: 0.0963635	total: 13s	remaining: 13.3s
494:	learn: 0.0960661	total: 13s	remaining: 13.3s
495:	learn: 0.0959297	total: 13.1s	remaining: 13.3s
496:	learn: 0.0957087	total: 13.1s	remaining: 13.2s
497:	learn: 0.0954142	total: 13.1s	remaining: 13.2s
498:	learn: 0.0951951	total: 13.1s	remaining: 13.2s
499:	learn: 0.0948897	total: 13.2s	remaining: 13.2s
500:	learn: 0.0947161	total: 13.2s	remaining: 13.1s
501:	learn: 0.0945

641:	learn: 0.0721412	total: 17.1s	remaining: 9.51s
642:	learn: 0.0719634	total: 17.1s	remaining: 9.48s
643:	learn: 0.0718788	total: 17.1s	remaining: 9.46s
644:	learn: 0.0718047	total: 17.1s	remaining: 9.43s
645:	learn: 0.0716543	total: 17.2s	remaining: 9.4s
646:	learn: 0.0716099	total: 17.2s	remaining: 9.37s
647:	learn: 0.0715016	total: 17.2s	remaining: 9.35s
648:	learn: 0.0713485	total: 17.2s	remaining: 9.32s
649:	learn: 0.0711619	total: 17.3s	remaining: 9.29s
650:	learn: 0.0710625	total: 17.3s	remaining: 9.26s
651:	learn: 0.0709849	total: 17.3s	remaining: 9.24s
652:	learn: 0.0708604	total: 17.3s	remaining: 9.21s
653:	learn: 0.0707711	total: 17.4s	remaining: 9.18s
654:	learn: 0.0706270	total: 17.4s	remaining: 9.16s
655:	learn: 0.0705136	total: 17.4s	remaining: 9.13s
656:	learn: 0.0704003	total: 17.4s	remaining: 9.11s
657:	learn: 0.0703351	total: 17.5s	remaining: 9.08s
658:	learn: 0.0702848	total: 17.5s	remaining: 9.06s
659:	learn: 0.0701296	total: 17.5s	remaining: 9.03s
660:	learn: 0

802:	learn: 0.0572025	total: 21.4s	remaining: 5.25s
803:	learn: 0.0571089	total: 21.4s	remaining: 5.22s
804:	learn: 0.0570577	total: 21.4s	remaining: 5.19s
805:	learn: 0.0569887	total: 21.5s	remaining: 5.17s
806:	learn: 0.0569057	total: 21.5s	remaining: 5.14s
807:	learn: 0.0568231	total: 21.5s	remaining: 5.11s
808:	learn: 0.0567560	total: 21.5s	remaining: 5.08s
809:	learn: 0.0566693	total: 21.5s	remaining: 5.05s
810:	learn: 0.0565591	total: 21.6s	remaining: 5.02s
811:	learn: 0.0564381	total: 21.6s	remaining: 4.99s
812:	learn: 0.0563392	total: 21.6s	remaining: 4.97s
813:	learn: 0.0562708	total: 21.6s	remaining: 4.94s
814:	learn: 0.0562271	total: 21.6s	remaining: 4.91s
815:	learn: 0.0561423	total: 21.7s	remaining: 4.89s
816:	learn: 0.0560638	total: 21.7s	remaining: 4.86s
817:	learn: 0.0560166	total: 21.7s	remaining: 4.83s
818:	learn: 0.0559292	total: 21.8s	remaining: 4.81s
819:	learn: 0.0558565	total: 21.8s	remaining: 4.78s
820:	learn: 0.0558355	total: 21.8s	remaining: 4.76s
821:	learn: 

963:	learn: 0.0476224	total: 25.6s	remaining: 957ms
964:	learn: 0.0475864	total: 25.6s	remaining: 930ms
965:	learn: 0.0475700	total: 25.7s	remaining: 904ms
966:	learn: 0.0475225	total: 25.7s	remaining: 877ms
967:	learn: 0.0474777	total: 25.7s	remaining: 851ms
968:	learn: 0.0474263	total: 25.8s	remaining: 824ms
969:	learn: 0.0473746	total: 25.8s	remaining: 797ms
970:	learn: 0.0473535	total: 25.8s	remaining: 771ms
971:	learn: 0.0473127	total: 25.8s	remaining: 744ms
972:	learn: 0.0472454	total: 25.9s	remaining: 718ms
973:	learn: 0.0471836	total: 25.9s	remaining: 691ms
974:	learn: 0.0471155	total: 25.9s	remaining: 664ms
975:	learn: 0.0470893	total: 25.9s	remaining: 638ms
976:	learn: 0.0470246	total: 26s	remaining: 611ms
977:	learn: 0.0469885	total: 26s	remaining: 585ms
978:	learn: 0.0469682	total: 26s	remaining: 558ms
979:	learn: 0.0468998	total: 26.1s	remaining: 532ms
980:	learn: 0.0468549	total: 26.1s	remaining: 505ms
981:	learn: 0.0467866	total: 26.1s	remaining: 478ms
982:	learn: 0.0467

In [41]:
print_classification_metrics(cbc, y_test, cbc.predict(X_test))

[[5 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 1 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 1 0 0 0 0 0 0]
 [0 0 0 0 0 9 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 2 2 1 0 0 0 0]
 [0 0 0 0 0 0 1 2 1 1 0 0 0]
 [0 0 0 0 0 0 2 0 3 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 3 0 0 0]
 [0 0 0 0 0 0 0 1 0 2 4 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           5       0.00      0.00      0.00         1
           6       0.50      0.33      0.40         3
           8       0.67      0.33      0.44         6
           9       0.00      0.00      0.00         5
          10       0.53      0.90      0.67        10
          11       0.29      0.33      0.31         6
          12       0.40      0.40      0.40         5
          13       0.50      0.50      0.50         6
          14       0.43      0.75      0.55         4
          15       0.80    

# Regression

### Подготовка данных

In [42]:
df = pd.read_csv('data/students_preprocessed.csv', sep = ',')
students_df = df[list(('school', 'sex', 'age', 'Pstatus', 'studytime', \
                                  'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'higher', \
                                  'internet', 'absences', 'G1', 'G2', 'G3'))]

In [43]:
X = students_df.drop('G3', axis = 1)
y = students_df.G3

In [44]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = 0.15)

In [45]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
def get_regression_metrics(estimator, y_test, y_pred):
    print(f"coefficient of determination: {estimator.score(X_train, y_train)}")
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

### DTR (base model)

In [47]:
%%time
tr = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
tr_base = GridSearchCV(tr, tree_params, cv=5).fit(X_train, y_train)
tr_base.best_params_

Wall time: 4.93 s


{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [48]:
get_regression_metrics(tr_base, y_test, tr_base.predict(X_test))

coefficient of determination: 0.9438384184406987
MSE: 2.864644211725739
RMSE: 1.6925259855392882
MAE: 0.8827402009754952


### CatBoostingRegressor

In [49]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)

0:	learn: 4.1249379	total: 1.96ms	remaining: 1.96s
1:	learn: 3.7725843	total: 4.36ms	remaining: 2.17s
2:	learn: 3.4217795	total: 6.26ms	remaining: 2.08s
3:	learn: 3.1841502	total: 8.15ms	remaining: 2.03s
4:	learn: 2.9506228	total: 10.5ms	remaining: 2.09s
5:	learn: 2.7777706	total: 12.4ms	remaining: 2.06s
6:	learn: 2.5801243	total: 15ms	remaining: 2.13s
7:	learn: 2.4013340	total: 17ms	remaining: 2.11s
8:	learn: 2.2700304	total: 18.8ms	remaining: 2.07s
9:	learn: 2.1614737	total: 20.6ms	remaining: 2.04s
10:	learn: 2.0817000	total: 22.6ms	remaining: 2.03s
11:	learn: 1.9837836	total: 24.4ms	remaining: 2.01s
12:	learn: 1.9042128	total: 26.3ms	remaining: 2s
13:	learn: 1.8170668	total: 28.3ms	remaining: 1.99s
14:	learn: 1.7620537	total: 29.9ms	remaining: 1.97s
15:	learn: 1.6999808	total: 32.5ms	remaining: 2s
16:	learn: 1.6650876	total: 35.5ms	remaining: 2.05s
17:	learn: 1.6118216	total: 37.4ms	remaining: 2.04s
18:	learn: 1.5763676	total: 39.1ms	remaining: 2.02s
19:	learn: 1.5317977	total: 40.7

187:	learn: 0.4232762	total: 456ms	remaining: 1.97s
188:	learn: 0.4219312	total: 458ms	remaining: 1.97s
189:	learn: 0.4204167	total: 460ms	remaining: 1.96s
190:	learn: 0.4199720	total: 464ms	remaining: 1.96s
191:	learn: 0.4178074	total: 466ms	remaining: 1.96s
192:	learn: 0.4168090	total: 469ms	remaining: 1.96s
193:	learn: 0.4145552	total: 471ms	remaining: 1.96s
194:	learn: 0.4096124	total: 473ms	remaining: 1.95s
195:	learn: 0.4090510	total: 475ms	remaining: 1.95s
196:	learn: 0.4069800	total: 478ms	remaining: 1.95s
197:	learn: 0.4039391	total: 481ms	remaining: 1.95s
198:	learn: 0.4019579	total: 486ms	remaining: 1.96s
199:	learn: 0.4019012	total: 487ms	remaining: 1.95s
200:	learn: 0.4008612	total: 492ms	remaining: 1.96s
201:	learn: 0.3978444	total: 495ms	remaining: 1.95s
202:	learn: 0.3948852	total: 497ms	remaining: 1.95s
203:	learn: 0.3943194	total: 499ms	remaining: 1.95s
204:	learn: 0.3922870	total: 502ms	remaining: 1.95s
205:	learn: 0.3898025	total: 504ms	remaining: 1.94s
206:	learn: 

359:	learn: 0.2053358	total: 909ms	remaining: 1.61s
360:	learn: 0.2041066	total: 912ms	remaining: 1.61s
361:	learn: 0.2039357	total: 914ms	remaining: 1.61s
362:	learn: 0.2031453	total: 916ms	remaining: 1.61s
363:	learn: 0.2030105	total: 919ms	remaining: 1.6s
364:	learn: 0.2016571	total: 922ms	remaining: 1.6s
365:	learn: 0.2004571	total: 925ms	remaining: 1.6s
366:	learn: 0.1997336	total: 927ms	remaining: 1.6s
367:	learn: 0.1991966	total: 929ms	remaining: 1.6s
368:	learn: 0.1990020	total: 932ms	remaining: 1.59s
369:	learn: 0.1980996	total: 934ms	remaining: 1.59s
370:	learn: 0.1979876	total: 936ms	remaining: 1.59s
371:	learn: 0.1971869	total: 938ms	remaining: 1.58s
372:	learn: 0.1970463	total: 940ms	remaining: 1.58s
373:	learn: 0.1970301	total: 942ms	remaining: 1.57s
374:	learn: 0.1955543	total: 943ms	remaining: 1.57s
375:	learn: 0.1948624	total: 944ms	remaining: 1.57s
376:	learn: 0.1943189	total: 946ms	remaining: 1.56s
377:	learn: 0.1935227	total: 948ms	remaining: 1.56s
378:	learn: 0.192

549:	learn: 0.1065381	total: 1.36s	remaining: 1.12s
550:	learn: 0.1060706	total: 1.37s	remaining: 1.11s
551:	learn: 0.1057744	total: 1.37s	remaining: 1.11s
552:	learn: 0.1052977	total: 1.37s	remaining: 1.11s
553:	learn: 0.1049135	total: 1.37s	remaining: 1.1s
554:	learn: 0.1047167	total: 1.38s	remaining: 1.1s
555:	learn: 0.1037412	total: 1.38s	remaining: 1.1s
556:	learn: 0.1030309	total: 1.38s	remaining: 1.1s
557:	learn: 0.1027154	total: 1.38s	remaining: 1.09s
558:	learn: 0.1025579	total: 1.39s	remaining: 1.09s
559:	learn: 0.1025426	total: 1.39s	remaining: 1.09s
560:	learn: 0.1021506	total: 1.39s	remaining: 1.09s
561:	learn: 0.1016805	total: 1.39s	remaining: 1.08s
562:	learn: 0.1015777	total: 1.39s	remaining: 1.08s
563:	learn: 0.1009434	total: 1.4s	remaining: 1.08s
564:	learn: 0.1007674	total: 1.4s	remaining: 1.07s
565:	learn: 0.1004582	total: 1.4s	remaining: 1.07s
566:	learn: 0.1004532	total: 1.4s	remaining: 1.07s
567:	learn: 0.1001122	total: 1.4s	remaining: 1.07s
568:	learn: 0.0997855

734:	learn: 0.0649977	total: 1.82s	remaining: 656ms
735:	learn: 0.0649839	total: 1.82s	remaining: 654ms
736:	learn: 0.0648589	total: 1.83s	remaining: 652ms
737:	learn: 0.0645584	total: 1.83s	remaining: 649ms
738:	learn: 0.0644785	total: 1.83s	remaining: 647ms
739:	learn: 0.0644235	total: 1.83s	remaining: 644ms
740:	learn: 0.0643872	total: 1.83s	remaining: 641ms
741:	learn: 0.0640740	total: 1.84s	remaining: 639ms
742:	learn: 0.0640564	total: 1.84s	remaining: 636ms
743:	learn: 0.0640000	total: 1.84s	remaining: 633ms
744:	learn: 0.0638943	total: 1.84s	remaining: 630ms
745:	learn: 0.0637865	total: 1.84s	remaining: 627ms
746:	learn: 0.0636865	total: 1.84s	remaining: 625ms
747:	learn: 0.0633719	total: 1.84s	remaining: 622ms
748:	learn: 0.0631704	total: 1.85s	remaining: 619ms
749:	learn: 0.0629627	total: 1.85s	remaining: 616ms
750:	learn: 0.0628507	total: 1.85s	remaining: 614ms
751:	learn: 0.0627350	total: 1.85s	remaining: 611ms
752:	learn: 0.0626389	total: 1.86s	remaining: 609ms
753:	learn: 

925:	learn: 0.0497126	total: 2.27s	remaining: 182ms
926:	learn: 0.0496991	total: 2.28s	remaining: 179ms
927:	learn: 0.0496659	total: 2.28s	remaining: 177ms
928:	learn: 0.0495504	total: 2.28s	remaining: 174ms
929:	learn: 0.0494897	total: 2.29s	remaining: 172ms
930:	learn: 0.0493863	total: 2.29s	remaining: 170ms
931:	learn: 0.0493394	total: 2.29s	remaining: 167ms
932:	learn: 0.0492864	total: 2.29s	remaining: 165ms
933:	learn: 0.0492075	total: 2.3s	remaining: 162ms
934:	learn: 0.0491826	total: 2.3s	remaining: 160ms
935:	learn: 0.0491487	total: 2.3s	remaining: 158ms
936:	learn: 0.0491245	total: 2.31s	remaining: 155ms
937:	learn: 0.0491124	total: 2.31s	remaining: 153ms
938:	learn: 0.0490205	total: 2.31s	remaining: 150ms
939:	learn: 0.0489817	total: 2.31s	remaining: 148ms
940:	learn: 0.0489170	total: 2.32s	remaining: 145ms
941:	learn: 0.0488620	total: 2.32s	remaining: 143ms
942:	learn: 0.0488373	total: 2.32s	remaining: 140ms
943:	learn: 0.0487741	total: 2.32s	remaining: 138ms
944:	learn: 0.0

In [50]:
get_regression_metrics(cbr, y_test, cbr.predict(X_test))

coefficient of determination: 0.9998993412860205
MSE: 2.9147856085615826
RMSE: 1.7072743214145707
MAE: 1.1527109756414697
